In [3]:
import os

import optuna
from optuna.trial import TrialState

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data

In [103]:
# determine the supported device
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

# convert a df to tensor to be used in pytorch
def df_to_tensor(df):
    device = get_device()
    return torch.from_numpy(df.values).float().to(device)

def split_df():
    df_X = pd.read_csv("bitcoin_train.csv",usecols=[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 24, 27, 28, 29, 32, 33, 34])
    df_y = pd.read_csv("bitcoin_train.csv",usecols=[37])
    X = df_to_tensor(df_X)
    y = df_to_tensor(df_y)
    return train_test_split(X, y, random_state=1234)

In [104]:
df_X = pd.read_csv("bitcoin_train.csv",usecols=[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 24, 27, 28, 29, 32, 33, 34])
df_y = pd.read_csv("bitcoin_train.csv",usecols=[37])
X = df_to_tensor(df_X)
y = df_to_tensor(df_y)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=1234)

In [102]:
X = pd.read_csv("bitcoin_train.csv",usecols=[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 24, 27, 28, 29, 32, 33, 34])
y = pd.read_csv("bitcoin_train.csv",usecols=[38])

def y_2d(x):
    if x == 1.0:
        return 0
    return 1
    
#y['second_label'] = y.apply(lambda x: y_2d(x[0]), axis = 1)

X = X.to_numpy()
y = y.to_numpy()

X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=1234)

ValueError: Usecols do not match columns, columns expected but not found: [38]

In [7]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.normal_(m.weight.data)
        nn.init.normal_(m.bias.data)

class Net(nn.Module):

    def __init__(self, sizes):
        super(Net, self).__init__()
        layers = []
        for i in range(len(sizes) - 1):
            layers.append(nn.Linear(sizes[i], sizes[i + 1]))
            layers.append(nn.Sigmoid())
        self.model = nn.Sequential(*layers)
        self.model.apply(init_weights)
        self.softmax = nn.Softmax(dim=0)

    def forward(self, x):
        return self.model(x)
        #pred = self.softmax(self.model.forward(x)).argmax()
        #if pred == 0:
        #    return torch.Tensor([1])
        #return torch.Tensor([0])
        
        

    def train(self, X_train, y_train, X_valid=None, y_valid=None,
          eta=0.25, num_epochs=10, isPrint=True):
        """
        Train the network with SGD 

        :param X_train: matrix of training features 
        :param y_train: matrix of vector-encoded labels 
        """
        criterion = nn.MSELoss()
        optimizer = optim.SGD(self.parameters(), lr=eta)
        # initialize shuffled indices 
        shuffled_inds = list(range(X_train.shape[0]))
        # loop over training epochs 
        for ep in range(1, num_epochs+1):

            # shuffle indices 
            np.random.shuffle(shuffled_inds)

            # loop over training examples 
            for ind in shuffled_inds:
                optimizer.zero_grad()
                output = self.forward(torch.from_numpy(X_train[ind]).float())
                loss = criterion(output, torch.from_numpy(y_train[ind]).float())
                loss.backward() 
                optimizer.step() 

            # print mean loss every 10 epochs if requested 
            if isPrint and (ep % 10) == 0:
                print("epoch {:3d}/{:3d}: ".format(ep, num_epochs), end="")
                print("  train loss: {:8.3f}".format(self.compute_loss(X_train, y_train)), end="")
                if X_valid is not None:
                    print("  validation loss: {:8.3f}".format(self.compute_loss(X_valid, y_valid)))
                else:
                    print("")

    def compute_loss(self, X, y):
        """
        compute average loss for given data set 
        
        :param X: matrix of features 
        :param y: matrix of vector-encoded labels 
        """
        loss = 0
        if len(X.shape) == 1:
            X = X[np.newaxis, :]
        if len(y.shape) == 1:
            y = y[np.newaxis, :]
        criterion = nn.MSELoss()
        for x, t in zip(X, y):
            output = self.forward(torch.from_numpy(x).float())
            loss += criterion(output, torch.from_numpy(t).float()).item()
        return loss / X.shape[0]
    

In [25]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.normal_(m.weight.data)
        nn.init.normal_(m.bias.data)

class Net(nn.Module):

    def __init__(self, sizes):
        super(Net, self).__init__()
        layers = []
        for i in range(len(sizes) - 1):
            layers.append(nn.Linear(sizes[i], sizes[i + 1]))
            layers.append(nn.Sigmoid())
        self.model = nn.Sequential(*layers)
        self.model.apply(init_weights)
        self.softmax = nn.Softmax(dim=0)

    def forward(self, x):
        return self.model(x)
        #pred = self.softmax(self.model.forward(x)).argmax()
        #if pred == 0:
        #    return torch.Tensor([1])
        #return torch.Tensor([0])
        
        

    def train(self, X_train, y_train, X_valid=None, y_valid=None,
          eta=0.25, num_epochs=10, isPrint=True):
        """
        Train the network with SGD 

        :param X_train: matrix of training features 
        :param y_train: matrix of vector-encoded labels 
        """
        criterion = nn.MSELoss()
        optimizer = optim.SGD(self.parameters(), lr=eta)
        # initialize shuffled indices 
        shuffled_inds = list(range(X_train.shape[0]))
        # loop over training epochs 
        for ep in range(1, num_epochs+1):

            # shuffle indices 
            np.random.shuffle(shuffled_inds)

            # loop over training examples 
            for ind in shuffled_inds:
                optimizer.zero_grad()
                output = self.forward(X_train[ind])
                loss = criterion(output,y_train[ind])
                loss.backward() 
                optimizer.step() 

            # print mean loss every 10 epochs if requested 
            if isPrint and (ep % 10) == 0:
                print("epoch {:3d}/{:3d}: ".format(ep, num_epochs), end="")
                print("  train loss: {:8.3f}".format(self.compute_loss(X_train, y_train)), end="")
                if X_valid is not None:
                    print("  validation loss: {:8.3f}".format(self.compute_loss(X_valid, y_valid)))
                else:
                    print("")

    def compute_loss(self, X, y):
        """
        compute average loss for given data set 
        
        :param X: matrix of features 
        :param y: matrix of vector-encoded labels 
        """
        loss = 0
        if len(X.shape) == 1:
            X = X[np.newaxis, :]
        if len(y.shape) == 1:
            y = y[np.newaxis, :]
        criterion = nn.MSELoss()
        for x, t in zip(X, y):
            output = self.forward(x)
            loss += criterion(output, t).item()
        return loss / X.shape[0]

In [27]:
model = Net([X_train.shape[1], 10, 10, 1])

In [28]:
model.train(X_train, y_train, X_valid, y_valid, eta=0.10, num_epochs = 100)

epoch  10/100:   train loss:    0.253  validation loss:    0.248
epoch  20/100:   train loss:    0.252  validation loss:    0.255
epoch  30/100:   train loss:    0.247  validation loss:    0.251
epoch  40/100:   train loss:    0.249  validation loss:    0.253
epoch  50/100:   train loss:    0.249  validation loss:    0.253
epoch  60/100:   train loss:    0.245  validation loss:    0.247
epoch  70/100:   train loss:    0.246  validation loss:    0.246
epoch  80/100:   train loss:    0.249  validation loss:    0.248
epoch  90/100:   train loss:    0.248  validation loss:    0.252
epoch 100/100:   train loss:    0.245  validation loss:    0.248


In [ ]:
## pred = model.forward(torch.from_numpy(X_valid[13]).float())
loss_1 = loss(torch.round(pred), torch.Tensor([1]))
loss_1.backward()

In [66]:
loss = nn.MSELoss()

In [114]:
def define_model(trial, in_features):
    '''
    Define NN model
    
    Inputs:
        trial: Optuna's Study object
        in_features (int): number of features
    '''
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = []

    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 4, 128)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.Sigmoid())
        p = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.5)
        layers.append(nn.Dropout(p))

        in_features = out_features
    layers.append(nn.Linear(in_features, CLASSES))
    #layers.append(nn.LogSoftmax(dim=1))
    layers.append(nn.Sigmoid())

    return nn.Sequential(*layers)

In [70]:
def define_model_b():
    '''
    Define NN model
    
    Inputs:
        trial: Optuna's Study object
        in_features (int): number of features
    '''
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = 1
    layers = []
    
    in_features = X_valid.shape[0]

    for i in range(n_layers):
        out_features = 8 #trial.suggest_int("n_units_l{}".format(i), 4, 128)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.Sigmoid())
        #p = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.5)
        #layers.append(nn.Dropout(p))

        in_features = out_features
    layers.append(nn.Linear(in_features, CLASSES))
    layers.append(nn.Sigmoid())

    return nn.Sequential(*layers)

In [71]:
DEVICE = get_device()
CLASSES = 2
model = define_model_b(X_train.shape[1]).to(DEVICE)

TypeError: define_model_b() takes 0 positional arguments but 1 was given

In [77]:
def objective(trial):

    # Generate the model.
    model = define_model(trial, NUM_FEATURES).to(DEVICE)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    # Training of the model.
    for epoch in range(EPOCHS):
        model.train()

        X_train, X_valid, y_train, y_valid = split_df()

        optimizer.zero_grad()
        output = model(X_train)
        loss = F.mse_loss(output, y_train)
        loss.backward()
        optimizer.step()

        # Validation of the model.
        model.eval()
        correct = 0
        #with torch.no_grad():
        #    for batch_idx, (data, target) in enumerate(valid_loader):
        #        
        #        output = model(X_valid)
        #        # Get the index of the max log-probability.
        #        pred = output.argmax(dim=1, keepdim=True)
        #        correct += pred.eq(target.view_as(pred)).sum().item()

        #accuracy = correct / min(len(valid_loader.dataset), N_VALID_EXAMPLES)
        output = model(X_valid)
        y_hat = output.detach().apply_(lambda x: round(x))
        correct = y_hat.eq(y_valid).sum().item()
        accuracy = correct / X_valid.shape[0]

        trial.report(accuracy, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return accuracy



In [53]:
criterion = nn.MSELoss()
criterion(output, y_train)

tensor(0.2463, grad_fn=<MseLossBackward0>)

In [59]:
y_hat = output.detach().apply_(lambda x: round(x))

In [66]:
y_hat.eq(y_train).sum().item() / y_train.shape[0]

0.5601703940362087

In [112]:
DEVICE = get_device()
CLASSES = 1
DIR = os.getcwd()
EPOCHS = 100
NUM_FEATURES = X_valid.shape[1]

In [115]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10, timeout=60)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-05-25 16:54:54,273] A new study created in memory with name: no-name-89f1965b-e5b7-49f7-8536-4fe60374fc52
[I 2022-05-25 16:54:55,380] Trial 0 finished with value: 0.5782747603833865 and parameters: {'n_layers': 3, 'n_units_l0': 34, 'dropout_l0': 0.3768938809309429, 'n_units_l1': 20, 'dropout_l1': 0.41453735681957093, 'n_units_l2': 59, 'dropout_l2': 0.3778615869451023, 'optimizer': 'SGD', 'lr': 0.0009460704260547728}. Best is trial 0 with value: 0.5782747603833865.
[I 2022-05-25 16:54:56,299] Trial 1 finished with value: 0.48562300319488816 and parameters: {'n_layers': 1, 'n_units_l0': 65, 'dropout_l0': 0.3245672090851371, 'optimizer': 'SGD', 'lr': 1.1097815723469417e-05}. Best is trial 0 with value: 0.5782747603833865.
[I 2022-05-25 16:54:57,242] Trial 2 finished with value: 0.5718849840255591 and parameters: {'n_layers': 1, 'n_units_l0': 75, 'dropout_l0': 0.41410364351330586, 'optimizer': 'Adam', 'lr': 5.018356152252206e-05}. Best is trial 0 with value: 0.5782747603833865.
[I 

Study statistics: 
  Number of finished trials:  10
  Number of pruned trials:  5
  Number of complete trials:  5
Best trial:
  Value:  0.5782747603833865
  Params: 
    n_layers: 3
    n_units_l0: 34
    dropout_l0: 0.3768938809309429
    n_units_l1: 20
    dropout_l1: 0.41453735681957093
    n_units_l2: 59
    dropout_l2: 0.3778615869451023
    optimizer: SGD
    lr: 0.0009460704260547728


In [88]:
df_X.columns

Index(['Daily_Change', 'Daily_Change_Ind', 'MACD', 'PROC_3', 'PROC_5',
       'PROC_10', 'wpr', 'sto_os', 'goog_trend_score', 'count', 'compound',
       'retweets_count', 'Daily_Change_Perc', 'Close/Last_Gold', 'Open_Gold',
       'Daily_Change_Gold', 'Close/Last_SP500', 'Open_SP500',
       'Daily_Change_SP500'],
      dtype='object')

In [107]:
df_X = pd.read_csv("bitcoin_train.csv")

In [108]:
df_X.columns

Index(['Unnamed: 0', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume',
       'Daily_Change', 'Daily_Change_Ind', 'MACD', 'PROC_3', 'PROC_5',
       'PROC_10', 'wpr', 'sto_os', 'goog_trend_score', 'count', 'compound',
       'retweets_count', 'likes_count', 'replies_count',
       'compound_weighted_replies', 'compound_weighted_likes',
       'compound_weighted_retweets', 'Daily_Change_Perc', 'Weekly_Change',
       'Weekly_Change_Perc', 'Close/Last_Gold', 'Open_Gold',
       'Daily_Change_Gold', 'Daily_Change_Perc_Gold', 'Increased_Gold',
       'Close/Last_SP500', 'Open_SP500', 'Daily_Change_SP500',
       'Daily_Change_Perc_SP500', 'Increased_SP500', 'label'],
      dtype='object')

In [100]:
len(df_X.columns)

38

In [109]:
df_X.head()

,Unnamed: 0,Date,Open,High,Low,Close,Volume,Daily_Change,Daily_Change_Ind,MACD,...,Open_Gold,Daily_Change_Gold,Daily_Change_Perc_Gold,Increased_Gold,Close/Last_SP500,Open_SP500,Daily_Change_SP500,Daily_Change_Perc_SP500,Increased_SP500,label
0,708,2017-08-28,4384.450195,4403.930176,4224.640137,4382.660156,1959330048,-0.219727,0.0,318.734756,...,1297.6,17.7,0.013641,1,2444.24,2447.35,-3.11,-0.001271,0,1.0
1,563,2017-01-31,920.958984,972.018982,920.958984,970.403015,164582000,50.020996,1.0,12.846431,...,1197.7,13.7,0.011439,1,2278.87,2274.02,4.85,0.002133,1,1.0
2,1390,2020-05-14,9271.329102,9793.268555,9255.035156,9733.721680,56426907637,463.734375,1.0,491.284984,...,1723.1,17.8,0.010330,1,2852.50,2794.54,57.96,0.020740,1,0.0
3,19,2014-11-10,362.265015,374.816010,357.561005,366.924011,30450100,3.660003,1.0,-5.684871,...,1176.8,-17.0,-0.014446,0,2038.26,2032.01,6.25,0.003076,1,1.0
4,944,2018-08-06,7062.939941,7166.549805,6890.540039,6951.799805,3925900000,-116.680175,0.0,55.940525,...,1214.4,-5.8,-0.004776,0,2850.40,2840.29,10.11,0.003559,1,0.0
